Frequently Bought Toghether Items

Use only if on Colab and there is a _COLAB folder in your Google Drive with necessary files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("drive/MyDrive/_COLAB")

Initialize graphs etc

In [ ]:
#Author: Everyone
import random
import networkx as nx
import node2vec
from gensim.models import Word2Vec
import os.path
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time
import math

#Nodo target
target = -1 ##modificato in seguito a random

#Grafo completo FBT
full_graph = nx.Graph()

#Grafo cluster selezionato
clustered_graph = nx.Graph()

#File di lettura
file_ = "Amazon0302"
file_name = file_ + ".txt"
#file_name = "grafo_esempio.txt"

#Grafo directed se True o uniderected se False
directed = True

#embedding hyperparameters
#default = 1, 1, 10, 80
p=1
q=1
num_walks=10
walk_length=80

saved = False
if(os.path.isfile(file_ + ".model")):
  saved = True

### LOAD GRAPH ###
print("loading graph " + file_name)

if directed:
    ##genera il grafo directed utilizzando nx.DiGraph
    full_graph = nx.read_edgelist(file_name, nodetype=int, create_using=nx.DiGraph)

else:
    ##genera il grafo undirected utilizzando nx.Graph
    full_graph = nx.read_edgelist(file_name)

nx.set_edge_attributes(full_graph, 1, name='weight')

print(full_graph)

print("finished")

Embedding using stanford's node2vec

In [ ]:
#Author:Savoia Emanuele
if(saved):
  print("Saved model found, loading file...")
  model = Word2Vec.load(file_ + ".model")
else:
  print("No saved model found, generating embedding...")
  G = node2vec.Graph(full_graph, directed, p, q)
  G.preprocess_transition_probs()
  walks = G.simulate_walks(num_walks, walk_length)
  walks = [list(map(str, walk)) for walk in walks]
  model = Word2Vec(walks, window=10, min_count=0, sg=1, workers=100)
  model.save(file_ + ".model")

Kmeans and sampled kmeans

In [ ]:
#Author:Savoia Emanuele
vectors = []
#initialize vector of points to be used in Kmeans
for p in range(full_graph.number_of_nodes()):
    vectors.append(model.wv[p])
points = np.array(vectors)

Run kmeans

In [ ]:
#Author: Savoia Emanuele

##Min tolerated sh score to proceed is: [best_score - max_error]
max_error = 0.025

#use sklearn implementation of kmeans++ combined with silhouette score to find best k and clustering
# if sampled is True use MiniBatchKMeans instead of kmeans
def sklearn_kmeans_find_k(points, sampled=True):
  start_time = time.time()
  #best clustering until this iteration
  best_clustering = []
  #array of scores used for plot
  scores = []
  #array of K associated with scores, used for plot
  k_tried = []
  #best silhouette until now
  best_score = -99999
  #mnimum k value
  min_k = int(math.sqrt(len(points)))*4
  for k in range(min_k, len(points), 15):
    print("trying kmeans with k = " + str(k))
    score = -1.0
    clustering = []
    if sampled:
      #choose sample size based on k and vector size
      sample_size = int(k*1.75)
      if(sample_size < 1024 and len(points) >= 1024):
        sample_size = 1024
      elif(sample_size > len(points)):
        sample_size = len(points)
      #run kmeans++ with mini batches
      kmeans = MiniBatchKMeans(n_clusters=k, init='k-means++', batch_size=sample_size, n_init='auto')
      clustering = kmeans.fit_predict(points)      
    else:
      #run kmeans++
      kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto')
      clustering = kmeans.fit_predict(points)
    #calculate silhouette from sample
    score = silhouette_score(points, clustering, sample_size=min_k*10)
    print("> Current silhouette score:   " + str(score))
    print("> Last best silhouette score: " + str(best_score))
    #update results and check if within parameters
    if score >= best_score - max_error:
      best_clustering = clustering
      best_score = score
      scores.append(score)
      k_tried.append(k)
    else:
      print("Time since start: " + str(time.time() - start_time))
      return best_clustering, scores, k_tried

In [ ]:
#Author: Savoia Emanuele
best_c_sampled, scores_sampled, k_tried_sampled = sklearn_kmeans_find_k(points)
print(best_c_sampled)
best_c_unsampled, scores_unsampled, k_tried_unsampled = sklearn_kmeans_find_k(points, False)
print(best_c_sampled)

In [ ]:
#Author: Savoia Emanuele
clustered_sampled = []
clustered_unsampled = []

#Nodo target
target = random.randrange(full_graph.number_of_nodes())
print("Target node is: " + str(target))

target_cluster_sampled = best_c_sampled[target]
target_cluster_unsampled = best_c_unsampled[target]

for p in range(len(points)):
    if best_c_sampled[p] == target_cluster_sampled:
        clustered_sampled.append(p)
    if best_c_unsampled[p] == target_cluster_unsampled:
        clustered_unsampled.append(p)

clustered_graph_s = full_graph.subgraph(clustered_sampled)
clustered_graph_u = full_graph.subgraph(clustered_unsampled)

print("Clustered graph: " + str(clustered_graph_s))
#Remove nodes that are not connected
if(directed):
  u = clustered_graph_s.to_undirected()
  nodes = nx.node_connected_component(u, target)
  clustered_graph_s = clustered_graph_s.subgraph(nodes)
else:
  clustered_graph_s = nx.node_connected_component(clustered_graph_s, target)

print("Clustered graph's (weakly) connected component with target node: " + str(clustered_graph_s))

print("Clustered graph unsampled: " + str(clustered_graph_u))
#Remove nodes that are not connected
if(directed):
  u = clustered_graph_u.to_undirected()
  nodes = nx.node_connected_component(u, target)
  clustered_graph_u = clustered_graph_u.subgraph(nodes)
else:
  clustered_graph_u = nx.node_connected_component(clustered_graph_u, target)

print("Unsampled clustered graph's (weakly) connected component with target node: " + str(clustered_graph_u))

#########CHOOSE GRAPH WITH MORE NODES##############
clustered_graph = clustered_graph_u
if(len(clustered_graph_s.nodes) > len(clustered_graph_u.nodes)):
  clustered_graph = clustered_graph_s

Valutazione nodi (usare clustered_graph)

In [ ]:
#Author:Vendramin Riccardo
# FUNCTIONS DEFINITION

# Neighbor
def n(v, edges):
    neighbors = set()
    for edge in edges:
        if v in edge:
            neighbors.update(edge)

    neighbors.discard(v)

    return list(neighbors)

# Generate undirected with edge = both arcs
def create_undirected_graph_from_directed(directed_graph):
    undirected_graph = nx.Graph()

    for edge in directed_graph.edges():
        if directed_graph.has_edge(edge[0], edge[1]) and directed_graph.has_edge(edge[1], edge[0]):
            undirected_graph.add_edge(edge[0], edge[1])

    return undirected_graph

# Bron-Kerbosch algorithm with pivot
def BronKerbosch(R, P, X, edges, cliques):
    if not P and not X:
        # P and X are both empty, report R as a maximal clique
        cliques.append(R)
        return

    # Choose a pivot vertex u in P ⋃ X
    pivot = (set(P) | set(X)).pop()

    for v in set(P) - set(n(pivot, edges)):
        # Recursively explore the neighborhood of v
        BronKerbosch(R + [v], list(set(P) & set(n(v, edges))), list(set(X) & set(n(v, edges))), edges, cliques)

        # Remove v from P and add it to X
        P.remove(v)
        X.append(v)

# Max clique
def maxClique(cliques):
    return max(cliques, key=len, default=[])

# Find cliques of target node
def nodeCliques(all_cliques, selected_node):
    node_cliques = []
    for clique in all_cliques:
        if selected_node in clique:
            node_cliques.append(clique)
    return node_cliques

# Compute clustering coefficient for nodes
def runNodeCC(graph):
    node_cluster_coefs = nx.clustering(graph)
        
    return node_cluster_coefs

# Find clique with best cc
def bestClique(node_cliques, node_cluster_coefs):
    max_cc = 0
    best_clique = []
    for clique in node_cliques:
        avg_cc = 0;
        for node in clique:
            avg_cc += node_cluster_coefs[node]

        print("clique:  " + str(clique))
        print("clique cc:  " + str(avg_cc/len(clique)))
        
        if avg_cc/len(clique) > max_cc:
            best_clique = clique
    return best_clique

In [ ]:
#Author:Vendramin Riccardo

# Find target best clique
all_cliques = []
selected_node = 12
clustered_graph = create_undirected_graph_from_directed(clustered_graph)
BronKerbosch([], list(clustered_graph.nodes), [], clustered_graph.edges, all_cliques)
#print(all_cliques)
node_cliques = nodeCliques(all_cliques, selected_node)
print("Cliques nodo:")
print(node_cliques)
print("Best clique")
print(bestClique(node_cliques, runNodeCC(clustered_graph)))

In [ ]:
#Tentativi con rimozioni casuali
# Numero di nodi da rimuovere ad ogni iterazione
n_nodes_to_remove = 10

# Numero totale di iterazioni
n_iterations = 100

# Copia del grafo iniziale
graph_to_modify = graph.copy()

#results
results = []

for iteration in range(n_iterations):
    # Escludere il target
    filtered_nodes = [element for element in list(graph_to_modify.nodes()) if element != selected_node]
    
    # Rimuovi n nodi casuali dal grafo
    nodes_to_remove = random.sample(filtered_nodes, n_nodes_to_remove)
    updated_graph = graph_to_modify.copy()
    updated_graph.remove_nodes_from(nodes_to_remove)

    # Stampa o elabora i risultati desiderati
    print(f"Iteration {iteration + 1}: Removed nodes {nodes_to_remove}")

    all_cliques = []
    updated_graph = create_undirected_graph_from_directed(updated_graph)
    BronKerbosch([], list(updated_graph.nodes), [], updated_graph.edges, all_cliques)
    node_cliques = nodeCliques(all_cliques, selected_node)
    print("Cliques nodo:")
    print(node_cliques)
    best_clique = bestClique(node_cliques, runNodeCC(updated_graph))
    print("Best clique")
    print(best_clique)
    results.append(best_clique)
    
    # Aggiorna il grafo originale per la prossima iterazione
    #graph_to_modify = updated_graph.copy()


In [ ]:
#Author:Vendramin Riccardo
# Plot results

def generate_histogram(array_of_arrays):
    # Convert inner arrays to tuples and use Counter
    element_counts = Counter(tuple(inner_array) for inner_array in array_of_arrays)

    # Extract elements and their counts
    elements = list(element_counts.keys())
    counts = list(element_counts.values())

    # Plot the histogram
    plt.bar(range(len(elements)), counts, align='center')
    plt.xticks(range(len(elements)), elements)
    plt.xlabel('Best clique')
    plt.ylabel('Count')
    plt.title('Histogram of results obtained')
    
    # Annotate each bar with its count
    for i, count in enumerate(counts):
        plt.text(i, count + 0.1, str(count), ha='center', va='bottom')
    
    plt.show()

generate_histogram(results)